In [56]:
from sympy import *
import numpy as np

D = 2

## Definiendo simbolos
t, r, theta, phi, global_phi = symbols('t, r, \theta, \\varphi, global_phi')
dt, dr, dtheta, dphi = symbols('dt dr dtheta dphi')     
l0, l1, l2, l3 = symbols("l_0 l_1 l_2 l_3")
l, lamb, p, q, n, L = symbols("l lamb p q n L")

deldel = Function("DELDEL")
delf = Function("DELF")
box = Function("\\Box")

def getFuncName(i):
    if i==0:
        return alfa.name
    if i==1:
        return F.name
    if i==2:
        return G.name
    
    
if D == 1:
    ndmat = np.array([[[symbols("a") for _x in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, r)
    F = Function("F")(t, r)
    G = Function("G")(t, r)
elif D == 2:
    ndmat = np.array([[[[symbols("a") for _x in range(4)] for _y in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, theta, r)
    F = Function("F")(t, theta, r)
    G = Function("G")(t, theta, r)
elif D == 3:
    ndmat = np.array([[[[[symbols("a") for _x in range(4)] for _y in range(4)] for _z in range(4)] for _w in range(4)] for _i in range(3)])
    alfa = Function("a")(t, phi, theta, r)
    F = Function("F")(t, phi, theta, r)
    G = Function("G")(t, phi, theta, r)


symbolsString = ""
if D == 1:
    for _f in range(3):
        for _t in range(4):
            for _r in range(4):
                ndmat[_f][_t][_r] = symbols(getFuncName(_f)+str(_t)+str(_r))
                symbolsString += str(ndmat[_f][_t][_r])+" "
elif D == 2:
    for _f in range(3):
        for _t in range(4):
            for _theta in range(4):
                for _r in range(4):
                    ndmat[_f][_t][_theta][_r] = symbols(getFuncName(_f)+str(_t)+str(_theta)+str(_r))
                    symbolsString += str(ndmat[_f][_t][_theta][_r])+" "
elif D == 3:
    for _f in range(3):
        for _t in range(4):
            for _phi in range(4):
                for _theta in range(4):
                    for _r in range(4):
                        ndmat[_f][_t][_phi][_theta][_r] = symbols(getFuncName(_f)+str(_t)+str(_phi)+str(_theta)+str(_r))
                        symbolsString += str(ndmat[_f][_t][_phi][_theta][_r])+" "


In [57]:
import sys
sys.version

'3.8.10 | packaged by conda-forge | (default, May 11 2021, 06:39:48) \n[Clang 11.1.0 ]'

In [58]:
def DELDEL(f, g):
    #canon = -f.diff(t)*g.diff(t) + 1/l1**2*f.diff(r)*g.diff(r) + 1/l1**2*f.diff(theta)*g.diff(theta) + 1/l2**2*f.diff(phi)*g.diff(phi)
    canon = -f.diff(t)*g.diff(t) + f.diff(r)*g.diff(r) + f.diff(theta)*g.diff(theta) + f.diff(phi)*g.diff(phi)
    return canon

def BOX(f):
    canon = -f.diff(t, 2) + (f.diff(r, 2) + f.diff(theta, 2) + f.diff(phi, 2))
    #canon = -f.diff(t, 2) + 1/l1**2*f.diff(r, 2) + 1/l1**2*f.diff(theta, 2) + 1/l2 **2*f.diff(phi, 2)
    return canon

def DELf(f, g):
    canon = DELDEL(f, g) + f*BOX(g)

    #canon = -(f*g.diff(t)).diff(t) + (f*g.diff(r)).diff(r)+ (f*g.diff(theta)).diff(theta) + (f*g.diff(phi)).diff(phi)
    #canon = -(f*g.diff(t)).diff(t) + 1/l1**2*(f*g.diff(r)).diff(r)+ 1/l1**2*(f*g.diff(theta)).diff(theta) + 1/l2**2*(f*g.diff(phi)).diff(phi)
    return canon


with evaluate(True):
    motion_alfa = p*(alfa.diff(theta, 2)+alfa.diff(r, 2))-alfa.diff(t)
    motion_F = 0
    motion_G = 0
    
display(motion_alfa)
display(motion_F)
display(motion_G)

p*(Derivative(a(t, heta, r), (heta, 2)) + Derivative(a(t, heta, r), (r, 2))) - Derivative(a(t, heta, r), t)

0

0

In [59]:
def expandToDerivatives(func):
    p, q, s, b = map(Wild, 'pqsb')
    res = func.replace(deldel(p, q)*s, lambda p, q, s : DELDEL(p, q)*s)
    res = res.replace(delf(p, q)*s, lambda p, q, s : DELf(p, q)*s)
    return res.replace(box(p)*s, lambda p, s : BOX(p)*s)

In [60]:
res3a = (expandToDerivatives(motion_alfa))
display("Alfa done.")


'Alfa done.'

In [61]:
#display(res3a)
#display(res3f)
#display(res3g)

In [62]:
def getDelta(i):
    if i == t:
        return dt
    if r == i:
        return dr
    if theta == i:
        return dtheta
    if phi == i:
        return dphi

def getIndex(i):
    if t == i:
        return 0
    if phi == i:
        return 1
    if theta == i:
        return 2
    if r == i:
        return 3

def getFuncIndex(f):
    if f==F:
        return 1
    if f==G:
        return 2
    if f==alfa:
        return 0
    
def isThis(s, ix):
    if ix != None and s in ix.keys():
        return ix[s]
    else: 
        return 0
#http://www.iitg.ac.in/gtm/files/chap2.pdf
#https://www.uio.no/studier/emner/matnat/math/MAT-INF1100/h07/undervisningsmateriale/kap7.pdf

# 3: +1
# 2:  0
# 1: -1
# 0: -2

def getDiscreteSymbol(ndmat, f, ix=None):
    if D==3:
        canon = ndmat[getFuncIndex(f)][2+isThis(t, ix)][2+isThis(phi, ix)][2+isThis(theta, ix)][2+isThis(r, ix)]
    elif D==2:
        canon = ndmat[getFuncIndex(f)][2+isThis(t, ix)][2+isThis(theta, ix)][2+isThis(r, ix)]
    elif D==1:
        canon = ndmat[getFuncIndex(f)][2+isThis(t, ix)][2+isThis(r, ix)]
    return canon
    
def FOD(f, i, wrt):
    
    canon = (getDiscreteSymbol(ndmat, f, ix={i:1}) - getDiscreteSymbol(ndmat, f, ix={i:-1}))/(2*getDelta(i))
    return canon

def SOD(f, i, j, wrt):
    if i==j:
        if f!=wrt and f!=G:
            canon = 0 
        elif (f==wrt and i==t) or i != t:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:1}) - 2*getDiscreteSymbol(ndmat, f, ix={i:0}) + getDiscreteSymbol(ndmat, f, ix={i:-1}))/(getDelta(i)**2)


    else:
        if i != t and t != j:
            canon = (getDiscreteSymbol(ndmat, f, ix={i:1, j:1}) + getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) )/(4*getDelta(i)*getDelta(j))
        #backward difference
        elif (j==t):
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:1, j:-1}) - getDiscreteSymbol(ndmat, f, ix={i:0, j:-1}))/(getDelta(j))))
            
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
        elif (i == t):
            #canon = (1/(2*getDelta(i)*getDelta(j)))*(getDiscreteSymbol(ndmat, f, ix={i:1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:1, j:-1})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:0})-getDiscreteSymbol(ndmat, f, ix={i:-1, j:-1}))
            canon = (1/getDelta(i))*(((getDiscreteSymbol(ndmat, f, ix={i:0, j:1})-getDiscreteSymbol(ndmat, f, ix={i:0, j:0}))/(getDelta(j))) - ((getDiscreteSymbol(ndmat, f, ix={i:-1, j:1}) - getDiscreteSymbol(ndmat, f, ix={i:-1, j:0}))/(getDelta(j))))
            
        else:
            print("what")
    return canon

def discreti(arg, wrt):
    if len(arg.variables) == 1:
        return FOD(arg.expr, arg.variables[0], wrt)
    elif len(arg.variables) == 2:
        return SOD(arg.expr, arg.variables[0], arg.variables[1], wrt)
    else:
        return None

In [63]:
def discretizeVariables(func):
    res = func.replace(r, r*dr)
    res = res.replace(theta, theta*dtheta)
    res = res.replace(phi, global_phi*dphi)
    return res.replace(t, l*dt)

def discretizeDerivatives(func, wrt):
    res = func.replace(lambda arg: arg.is_Derivative, lambda arg: discreti(arg, wrt))
    res = res.replace(F, getDiscreteSymbol(ndmat, F))
    res = res.replace(G, getDiscreteSymbol(ndmat, G))
    return res.replace(alfa, getDiscreteSymbol(ndmat, alfa))

In [64]:
res7a = discretizeDerivatives(res3a, alfa)
res7a = discretizeVariables(res7a)


In [65]:
expra = ccode(res7a).replace("\\", "")


In [66]:
def getSymbolsUsed(expr):
    syms = ""
    for _y in expr.free_symbols:
        syms += str(_y)+" "
    return syms[:-1]

In [67]:
symma = getSymbolsUsed(res7a)
print("Symbols for a: \n [", symma, "]")



Symbols for a: 
 [ p dtheta dt a232 a223 a122 a212 a322 a221 a222 dr ]


In [68]:
#import pyximport
#pyximport.install()
from sage.all import *
import sage as sg

var(symma.replace("\\", ""))
var(symbolsString.replace("\\", ""))
var("E4 E5 E6 r theta global_phi")

#pi_4 = E4*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi)
#pi_5 = E5*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi)
#pi_6 = E6*sin(dr*r)*sin(dtheta*theta)*sin(dphi*global_phi)

dAdt = "0"
#dAdt = "0"

#dFdt = "0 + pi_5"
#dGdt = "p/L + pi_6"

def solveWithSage(expr, symbols, x):
    wata = eval(expr)
    resa = sg.symbolic.relation.solve(wata, x)
    try:
        return resa[0].full_simplify()
    except:
        try:
            return resa.full_simplify()
        except:
            return resa

In [69]:
def getDerivative(eq, var):
    if type(eq) == int:
        return 0
    else:
        try:
            return eq.derivative(var)
        except:
            return 0


"""def replaceOtherFunctionsNegative(expr, func, func1, func2, ofunc1dt, ofunc2dt):
    auxofun1 = eval(func1+"2222") - 2.0*dt*ofunc1dt 
    auxofun2 = eval(func2+"2222") - 2.0*dt*ofunc2dt 
    
    firsta = expr.subs({eval(func1+"0222") : auxofun1})
    firsta = firsta.subs({eval(func2+"0222") : auxofun2})
    firstaa = sg.symbolic.relation.solve(firsta, eval(func+"3222"))
    try:
        return firstaa[0]#.collect_common_factors().full_simplify().collect_common_factors()
    except:
        return firstaa
""";

In [70]:
from sage.misc.latex import Latex
ra=[]
if D==3:
    ra = solveWithSage(expra, symma, a3222)
if D==2:
    ra = solveWithSage(expra, symma, a322)
if D==1:
    ra = solveWithSage(expra, symma, a32)
#lates = Latex()
#lates(ra)

In [71]:
ra

a322 == (a122*dr^2*dtheta^2 + 2*((a212 - 2*a222 + a232)*dr^2*dt + (a221 - 2*a222 + a223)*dt*dtheta^2)*p)/(dr^2*dtheta^2)

In [72]:
#newra = replaceOtherFunctionsNegative(ra, "a", "F", "G", eval(dFdt), eval(dGdt))
#print(newra)

In [74]:
def nops(expr):
    count = 0
    string = ccode(expr._sympy_())
    count += string.count("*")
    count += string.count("^")
    count += string.count("-")
    count += string.count("+")
    count += string.count("/")
    return count

print(nops(ra))


18


In [75]:
 import re
def i2i(arg, index):
    if index == "0":
        return "("+arg+"-2)"
    if index == "1":
        return "("+arg+"-1)"
    if index == "2":
        return "("+arg+")"
    if index == "3":
        return "("+arg+"+1)"
def i2iTime(arg, index):
    if index == "0":
        return "(tm2)"
    if index == "1":
        return "(tm1)"
    if index == "2":
        return "(t)"
    if index == "3":
        return "(tp1)"    
def getCudaArray(ids):
    res = ""
    args = ["t", "threadIdx.z", "threadIdx.y", "threadIdx.x"]
    sizes = ["L", "O", "N", "M"]  
    while len(ids)!=4:
        ids = ids[0]+'2'+ids[1:]
    for i, c in enumerate(ids):
        if args[i] == "t":
            res += i2iTime(args[i], c)+","
        else:
            res += i2i(args[i], c)+","
        #for j,s in enumerate(sizes[i+1:]):
            #res+="*"+s
        #res+="+"
    return res[:-1]
def getCArray(ids):
    res = ""
    args = ["t", "phi", "theta", "r"]
    sizes = ["L", "O", "N", "M"]  
    while len(ids)!=4:
        ids = ids[0]+'2'+ids[1:]
    for i, c in enumerate(ids):
        if args[i] == "t":
            res += i2iTime(args[i], c)+","
        else:
            res += i2i(args[i], c)+","
        #for j,s in enumerate(sizes[i+1:]):
            #res+="*"+s
        #res+="+"
    return res[:-1]



def getExpressionInCuda(expr):
    
    traduc = ccode(expr)

    if D==3:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCudaArray(traduc[i+1:i+5]) +")]"+ traduc[i+5:]
    elif D==2:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCudaArray(traduc[i+1:i+4]) +")]"+ traduc[i+4:]
    elif D==1:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCudaArray(traduc[i+1:i+3]) +")]"+ traduc[i+3:]
    traduc = traduc.replace("a[I(", "shmem[CI(0,")
    traduc = traduc.replace("F[I(", "shmem[CI(1,")
    traduc = traduc.replace("G[I(", "shmem[CI(2,")
    
    traduc = traduc.replace("threadIdx.x+1", "tidxp")
    traduc = traduc.replace("threadIdx.x-1", "tidxm")
    traduc = traduc.replace("threadIdx.y+1", "tidyp")
    traduc = traduc.replace("threadIdx.y-1", "tidym")
    traduc = traduc.replace("threadIdx.z+1", "tidzp")
    traduc = traduc.replace("threadIdx.z-1", "tidzm")    


    return traduc

def getExpressionInC(expr):
    
    traduc = ccode(expr)

    if D==3:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+5]) +")]"+ traduc[i+5:]
    elif D==2:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+4]) +")]"+ traduc[i+4:]
    elif D==1:
        indexes = [m.start() for m in re.finditer(r'[a-zA-Z]\d\d', traduc)]
        for i in reversed(indexes):
            traduc = traduc[:i+1]+"[I("+ getCArray(traduc[i+1:i+3]) +")]"+ traduc[i+3:]
    #traduc = traduc.replace("a[I(", "shmem[CI(0,")
    #traduc = traduc.replace("F[I(", "shmem[CI(1,")
    #traduc = traduc.replace("G[I(", "shmem[CI(2,")
    
    #traduc = traduc.replace("threadIdx.x+1", "tidxp")
    #traduc = traduc.replace("threadIdx.x-1", "tidxm")
    #traduc = traduc.replace("threadIdx.y+1", "tidyp")
    #traduc = traduc.replace("threadIdx.y-1", "tidym")
    #traduc = traduc.replace("threadIdx.z+1", "tidzp")
    #traduc = traduc.replace("threadIdx.z-1", "tidzm")    


    return traduc

In [76]:
expr_a = getExpressionInC(ra._sympy_())
expr_a


'a[I((tp1),(phi),(theta),(r))] == (a[I((tm1),(phi),(theta),(r))]*pow(dr, 2)*pow(dtheta, 2) + 2*p*(pow(dr, 2)*dt*(a[I((t),(phi),(theta-1),(r))] - 2*a[I((t),(phi),(theta),(r))] + a[I((t),(phi),(theta+1),(r))]) + dt*pow(dtheta, 2)*(a[I((t),(phi),(theta),(r-1))] - 2*a[I((t),(phi),(theta),(r))] + a[I((t),(phi),(theta),(r+1))])))/(pow(dr, 2)*pow(dtheta, 2))'

In [81]:
expr_fnp = getExpressionInC(motion_F)
expr_gnp = getExpressionInC(motion_G)

In [82]:
expr_fnp

'0'

In [77]:
def writeDefinitionGPU(filename, template, expr, arr):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".cu"):
        #in that case, just append new declaration
        fin = open(filename+".cu", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#include \""+filename.split('/')[-1]+".cuh\"\n \
#include \"SharedMem.cuh\"\n"
    string = ''
    if arr == 'a':
        string = '0'
    elif arr == 'F':
        string = '1'
    else:
        string = '2'
    data = data + '\n' + template + """{
    int r = blockIdx.x*blockDim.x + threadIdx.x;
    int theta = blockIdx.y*blockDim.y + threadIdx.y;
    int phi = blockIdx.z*blockDim.z + threadIdx.z;
    int global_phi = phi + phi_offset;
    
    extern __shared__ REAL shmem[];
    if (r<M && theta<N && phi<O){
        fillSharedMemory(shmem, a, F, G, M, N, O, global_phi, r, theta, phi);
    }
    __syncthreads();
    if (r<M && theta<N && phi<O){
        int tidxp = threadIdx.x+1;
        int tidxm = threadIdx.x-1;
        
        int tidyp = threadIdx.y+1;
        int tidym = threadIdx.y-1;        
        
        int tidzp = threadIdx.z+1;
        int tidzm = threadIdx.z-1;
        
        """ + "\n\
        "+str(arr)+"[I(tp1, phi, theta, r)] = " + expr + ";\n\
    }\n\
}"
    
    fin = open(filename+".cu", "wt")
    fin.write(data)
    fin.close()
    
def writeDefinitionCPU(filename, template, expr, arr):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".cpp"):
        #in that case, just append new declaration
        fin = open(filename+".cpp", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#include <iostream>\n \
#include <unistd.h>\n \
#include <vector>\n \
#include <string>\n \
#include <fstream>\n \
#include <cmath>\n \
#include \"defines.h\"\n \
#include \""+filename.split('/')[-1]+".h\"\n"
    string = ''
    if arr == 'a':
        string = '0'
    elif arr == 'F':
        string = '1'
    else:
        string = '2'
    data = data + '\n' + template + """{
    return """ + expr + ";\n\
}"
    
    fin = open(filename+".cpp", "wt")
    fin.write(data)
    fin.close()

def writeDeclarationGPU(filename, template):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".cuh"):
        #in that case, just append new declaration
        fin = open(filename+".cuh", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#pragma once\n \
#include <iostream>\n \
#include <unistd.h>\n \
#include <vector>\n \
#include <string>\n \
#include <fstream>\n \
#include <cmath>\n \
#include \"defines.h\"\n \
using namespace std;\n"
    
    data = data + '\n' + template + ";"
    
    fin = open(filename+".cuh", "wt")
    fin.write(data)
    fin.close()

def writeDeclarationCPU(filename, template):
    import os.path
    data = ""
    #check if header already exist
    if os.path.isfile(filename+".h"):
        #in that case, just append new declaration
        fin = open(filename+".h", "rt")
        data = fin.read()
        fin.close()
    else:
        data = "//### THIS IS A GENERATED FILE - DO NOT MODIFY ### \n \
#pragma once\n \
#include <iostream>\n \
#include <unistd.h>\n \
#include <vector>\n \
#include <string>\n \
#include <fstream>\n \
#include <cmath>\n \
#include \"defines.h\"\n \
using namespace std;\n"
    
    data = data + '\n' + template + ";"
    
    fin = open(filename+".h", "wt")
    fin.write(data)
    fin.close()

def cleanOldFilesGPU(file):
    import os
    try:
        os.remove(file + ".cu")    
        os.remove(file + ".cuh")
    except:
        return "Already gone."
    
def cleanOldFilesCPU(file):
    import os
    try:
        os.remove(file + ".cpp")    
        os.remove(file + ".h")
    except:
        return "Already gone."
    
def writePairs(filename, funcname, expr, device='cpu'):
    if device == 'cpu':
        writeDeclarationCPU(filename, "REAL "+funcname+"(REAL *a, REAL *F, REAL *G, size_t t, size_t tm1, size_t tm2, size_t r, size_t theta, size_t phi, size_t M, size_t N, size_t O, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)")
        writeDefinitionCPU( filename, "REAL "+funcname+"(REAL *a, REAL *F, REAL *G, size_t t, size_t tm1, size_t tm2, size_t r, size_t theta, size_t phi, size_t M, size_t N, size_t O, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)", expr, funcname[-1])
    else:
        writeDeclarationGPU(filename, "__global__ void "+funcname+"(REAL *a, REAL *F, REAL *G, size_t l, size_t tp1, size_t t, size_t tm1, size_t tm2, size_t M, size_t N, size_t O, size_t phi_offset, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)")
        writeDefinitionGPU( filename, "__global__ void "+funcname+"(REAL *a, REAL *F, REAL *G, size_t l, size_t tp1, size_t t, size_t tm1, size_t tm2, size_t M, size_t N, size_t O, size_t phi_offset, REAL dt, REAL dr, REAL dtheta, REAL dphi, REAL l_1, REAL l_2, REAL lamb, int p, int q, int L)", expr, funcname[-1])


def findClosingPar(expr):
    let = 1
    i = 1
    #print(expr)
    while let > 0 and i<len(expr):
        if expr[i] == '(':
            let += 1
        elif expr[i] == ')':
            let -= 1
        i+=1
    return i-1

def replaceAllPow(expr):
    indexes = [m.start() for m in re.finditer('pow', expr)]
    for i in indexes[::-1]:
        open = i+4
        close = findClosingPar(expr[open:])+open
        exponent = expr[open:close].split(',')[-1]
        
        commas = [m.start() for m in re.finditer(',', expr[open:close])]
        #print(expr[open:close])
        argument = expr[open:open+commas[-1]]
        
        rep = ''
        for j in range(int(exponent)):
            rep += argument+'*'
        rep = rep[:-1]
        expr = expr[:i] +'(' + rep + ')'+ expr[close+1:]
    return expr  
        #print(expr)
        #return
        #print(rep)

In [78]:
expr_anp = replaceAllPow(expr_a)

#expr_newanp = replaceAllPow(expr_newa)
#expr_newfnp = replaceAllPow(expr_newf)
#expr_newgnp = replaceAllPow(expr_newg)

In [83]:
#cleanOldFilesGPU("../gpu/src/EquationAlfa")
#cleanOldFilesGPU("../gpu/src/EquationF")
#cleanOldFilesGPU("../gpu/src/EquationG")

cleanOldFilesCPU("../cpu/src/EquationAlfa")
cleanOldFilesCPU("../cpu/src/EquationF")
cleanOldFilesCPU("../cpu/src/EquationG")

#writePairs("../gpu/src/EquationAlfa", "computeNexta", expr_anp[63:])
writePairs("../cpu/src/EquationAlfa", "computeNexta", expr_anp[32:], 'cpu')

#writePairs("../gpu/src/EquationF", "computeNextF", expr_fnp[63:])
writePairs("../cpu/src/EquationF", "computeNextF", expr_fnp, 'cpu')

#writePairs("../gpu/src/EquationG", "computeNextG", expr_gnp[63:])
writePairs("../cpu/src/EquationG", "computeNextG", expr_gnp, 'cpu')


#writePairs("../gpu/src/EquationG", "computeSecondG", expr_newgnp[63:])